<a href="https://colab.research.google.com/github/molinajuandomingo24-max/labo2025v/blob/main/src/ensembles/420_ArbolesAzarosos_TareaHogar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. Ensembles de Arboles de Decision

Un arbol de decisión es un modelo débil, el aumento del poder predictivo proviene al ensamblar varios arboles de decisión.
<br> Si promedio n arboles identicos, el resultados es exactamente el mismo que utilizar un solo arbol, necesito PERTURBAR cada arbol para disponer de variablidad

la variabilidad provendrá de estas fuentes:


*   Perturbar el dataset
*   Perturbar el algoritmo del arbol
*   Perturbar el dataset y el algoritmo del arbol al mismo tiempo

Se verán estos tres algoritmos


*   Arboles Azarosos
*   Random Forest
*   Gradient Boosting of Decision Trees

#### 4.01 Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



---



## 4.02 Arboles Azarosos

Arboles Azarosos es el nombre de un algoritmo trivial (por favor NO confundir con Random Forest)
Qué tipo de perturbaciones se realizan en Arboles Azarosos
* Se perturba el dataset
* No se perturba el algoritmo, es siempre rpart original

Cada  arbolito de  Arboles Azarosos se entrena sobre un dataset perturbado,  que tiene exactamente la misma cantidad de registros pero solo un subconjunto de los atributos (campos)  del dataset, tomados al azar, de los originales.
<br> En esta primera corrida, se construira cada arbol en un dataset utilizando el 50% de los campos

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

In [ ]:
# cargo las librerias que necesito
require("data.table")
require("rpart")

Aqui debe cargar SU semilla primigenia

In [ ]:
PARAM <- list()
PARAM$semilla_primigenia <- 209353

# parametros  arbol
# entreno cada arbol con solo 50% de las variables variables
#  por ahora, es fijo
PARAM$feature_fraction <- 0.5

PARAM$rpart$cp <- -1
PARAM$rpart$minsplit <- 50
PARAM$rpart$minbucket <- 20
PARAM$rpart$maxdepth <- 6

# voy a generar 512 arboles,
#  a mas arboles mas tiempo de proceso y MEJOR MODELO,
#  pero ganancias marginales
PARAM$num_trees_max <- 512

In [ ]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento <- "exp4020"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [ ]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [ ]:
# defino los dataset de entrenamiento y aplicacion
dtrain <- dataset[foto_mes == 202107]
dfuture <- dataset[foto_mes == 202109]

# arreglo clase_ternaria por algun distraido ""
dfuture[, clase_ternaria := NA ]

In [ ]:
# Establezco cuales son los campos que puedo usar para la prediccion
# el copy() es por la Lazy Evaluation
campos_buenos <- copy(setdiff(colnames(dtrain), c("clase_ternaria")))

In [ ]:
# que tamanos de ensemble grabo a disco
grabar <- c(1, 2, 4, 8, 16, 32, 64, 128, 256, 384, 512)

In [ ]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]
# aqui se va acumulando la probabilidad del ensemble
tb_prediccion[, prob_acumulada := 0]

In [ ]:
set.seed(PARAM$semilla_primigenia) # Establezco la semilla aleatoria

In [ ]:
for (arbolito in seq(PARAM$num_trees_max) ) {
  message( arbolito, " ")
  qty_campos_a_utilizar <- as.integer(length(campos_buenos)
    * PARAM$feature_fraction)

  # elijo los campos al azar
  campos_random <- sample(campos_buenos, qty_campos_a_utilizar)

  # paso de un vector a un string con los elementos
  # separados por un signo de "+"
  # este hace falta para la formula
  campos_random <- paste(campos_random, collapse= " + ")

  # armo la formula para rpart
  formulita <- paste0("clase_ternaria ~ ", campos_random)

  # genero el arbol de decision
  modelo <- rpart(formulita,
    data= dtrain,
    xval= 0,
    control= PARAM$rpart
  )

  # aplico el modelo a los datos que no tienen clase
  prediccion <- predict(modelo, dfuture, type= "prob")

  tb_prediccion[, prob_acumulada := prob_acumulada + prediccion[, "BAJA+2"]]

  if (arbolito %in% grabar) {
    umbral_corte <- (1 / 40) * arbolito
    tb_prediccion[, Predicted := as.numeric(prob_acumulada > umbral_corte)]

    archivo_kaggle <- paste0(
        "KA420_",
        sprintf("%.3d", arbolito), # para que tenga ceros adelante
        ".csv"
      )

    # grabo el archivo
    fwrite( tb_prediccion[, list(numero_de_cliente, Predicted)],
      file= archivo_kaggle,
      sep= ","
    )

    # subida a Kaggle
    comando <- "kaggle competitions submit"
    competencia <- "-c labo-i-2025-ba-analista-sr"
    arch <- paste( "-f", archivo_kaggle)

    mensaje <- paste0("-m 'cp=", PARAM$rpart$cp, "  minsplit=", PARAM$rpart$minsplit, "  minbucket=", PARAM$rpart$minbucket, " maxdepth=", PARAM$rpart$maxdepth, "'" )
    linea <- paste( comando, competencia, arch, mensaje)
    salida <- system(linea, intern=TRUE)
    cat(salida)
  }
}


In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

In [ ]:
# --- Inicio del Código R ---

# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

# cargo las librerias que necesito
require("data.table")
require("rpart")
require("tools") # Para basename

# --- Parametros Fijos y Optimizables ---
PARAM <- list()
PARAM$semilla_primigenia <- 209353 # Su semilla primigenia

# Parametros para Arboles Azarosos
PARAM$feature_fraction <- 0.5  # Fracción de campos a usar en cada arbol
PARAM$num_trees_optimo <- 32   # Número de árboles fijo para la optimización

# Rangos para la optimización de hiperparámetros
# Los valores se eligen para crear arboles potencialmente 'overfitteros'
# (menor minsplit, mayor maxdepth) para el ensemble.
PARAM$minsplit_values <- c(50, 40, 30, 20)
PARAM$maxdepth_values <- c(6, 8, 10)
PARAM$cp_value <- -1 # cp se mantiene fijo

# --- Seteo de Directorios ---
# Carpeta de trabajo
# setwd("/content/buckets/b1/exp") # Esta línea se asume ejecutada en el script Python
experimento_base <- "exp420_opt_ensemble32"
# Dir.create solo si no existe, si existe se borran archivos viejos para un experimento limpio
dir.create(paste0("/content/buckets/b1/exp/", experimento_base), showWarnings=FALSE)
setwd(paste0("/content/buckets/b1/exp/", experimento_base))

# --- Lectura de Datos ---
# Se asume que el archivo ya fue descargado en el setup de Python
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# Defino los dataset de entrenamiento y aplicacion (futuro)
dtrain <- dataset[foto_mes == 202107]
dfuture <- dataset[foto_mes == 202109]

# Arreglo clase_ternaria por algun distraido ""
dfuture[, clase_ternaria := NA ]

# Establezco cuales son los campos que puedo usar para la prediccion
campos_buenos <- copy(setdiff(colnames(dtrain), c("clase_ternaria")))

# --- Búsqueda de Hiperparámetros ---
for (minsplit_actual in PARAM$minsplit_values) {
  for (maxdepth_actual in PARAM$maxdepth_values) {

    # 1. Definir los parámetros del arbolito para esta iteración
    # Se ajusta minbucket para ser consistente con minsplit (minbucket = minsplit/2)
    PARAM$rpart_current <- list(
      cp = PARAM$cp_value,
      minsplit = minsplit_actual,
      minbucket = as.integer(minsplit_actual / 2),
      maxdepth = maxdepth_actual
    )

    # Crear un subdirectorio para cada combinación de hiperparámetros
    exp_detalle <- paste0("m", minsplit_actual, "_d", maxdepth_actual)
    dir.create(exp_detalle, showWarnings=FALSE)
    setwd(exp_detalle)

    cat("--- Optimizando: minsplit=", minsplit_actual, ", maxdepth=", maxdepth_actual, " ---\n")

    # Inicializar para el ensemble
    tb_prediccion <- dfuture[, list(numero_de_cliente)]
    tb_prediccion[, prob_acumulada := 0]

    # Establezco la semilla aleatoria para reproducibilidad del ensemble
    set.seed(PARAM$semilla_primigenia)

    # Bucle para entrenar el ensemble de 32 arboles
    for (arbolito in seq(PARAM$num_trees_optimo) ) {

      qty_campos_a_utilizar <- as.integer(length(campos_buenos) * PARAM$feature_fraction)

      # Elijo los campos al azar
      # Usar una semilla específica del loop para reproducibilidad de la selección de campos
      set.seed(PARAM$semilla_primigenia + arbolito)
      campos_random <- sample(campos_buenos, qty_campos_a_utilizar)

      # Paso de un vector a un string con los elementos separados por un signo de "+"
      campos_random <- paste(campos_random, collapse= " + ")

      # Armo la formula para rpart
      formulita <- paste0("clase_ternaria ~ ", campos_random)

      # Genero el arbol de decision
      modelo <- rpart(formulita,
        data= dtrain,
        xval= 0,
        control= PARAM$rpart_current
      )

      # Aplico el modelo a los datos que no tienen clase
      prediccion <- predict(modelo, dfuture, type= "prob")

      # Acumulo la probabilidad
      tb_prediccion[, prob_acumulada := prob_acumulada + prediccion[, "BAJA+2"]]

      message(arbolito, " ")
    }

    # 2. Generar el archivo final con el ensemble de 32 árboles

    # El umbral de corte se calcula para un ensemble de N=32 arboles
    # En el original es (1 / 40) * arbolito, para 32 arboles es (1/40) * 32 = 0.8
    # Se utilizará el promedio de probabilidad, y el umbral de corte del archivo original
    # (que es el corte de la probabilidad promedio para Ganancia, tipicamente 1/40 ~ 0.025)

    # Se usará un umbral de corte simplificado (por ejemplo, 1/40) sobre la probabilidad promedio.
    # El ensemble finaliza en el arbolito 32.
    umbral_corte_promedio <- (1 / 40)

    # Se calcula la probabilidad promedio
    tb_prediccion[, prob_promedio := prob_acumulada / PARAM$num_trees_optimo]

    # Clasificación final
    tb_prediccion[, Predicted := as.numeric(prob_promedio > umbral_corte_promedio)]

    archivo_kaggle <- paste0(
      "KA420_Ensemble32_m", minsplit_actual, "_d", maxdepth_actual, ".csv"
    )

    # Grabo el archivo
    fwrite( tb_prediccion[, list(numero_de_cliente, Predicted)],
      file= archivo_kaggle,
      sep= ","
    )

    # 3. Subida a Kaggle para evaluación de la Ganancia
    comando <- "kaggle competitions submit"
    competencia <- "-c labo-i-2025-ba-analista-sr"
    arch <- paste( "-f", archivo_kaggle)

    mensaje <- paste0("-m 'minsplit=", minsplit_actual,
                      " minbucket=", PARAM$rpart_current$minbucket,
                      " maxdepth=", maxdepth_actual,
                      " Ensemble de ", PARAM$num_trees_optimo, " Arboles Azarosos'")

    linea <- paste( comando, competencia, arch, mensaje)
    # salida <- system(linea, intern=TRUE) # Descomentar para la ejecución real
    # cat(salida)
    cat("Comando Kaggle (simulado):", linea, "\n")

    # Volver al directorio base
    setwd(paste0("/content/buckets/b1/exp/", experimento_base))
  }
}

# --- Comparación y Conclusiones (Manual) ---
cat("\n--- Tarea de Comparación ---\n")
cat("Se han generado las predicciones para diferentes combinaciones de hiperparámetros de un ensemble de 32 Arboles Azarosos.\n")
cat("Los archivos .csv han sido (simuladamente) subidos a Kaggle para medir la Ganancia.\n")
cat("\n**Pasos para la Validación Final:**\n")
cat("1. Ejecutar este código para que los comandos 'kaggle competitions submit' se ejecuten (es necesario tener el entorno de Python/Kaggle configurado).\n")
cat("2. **Analizar los resultados en el leaderboard de Kaggle** para encontrar la combinación (minsplit, maxdepth) que maximiza la Ganancia.\n")
cat("3. **Comparar los hiperparámetros óptimos del ensemble de 32 (los que dan la mayor Ganancia en Kaggle) con los hiperparámetros óptimos de un árbol único** (que típicamente son más 'podados', con mayor minsplit y menor maxdepth).\n")
cat("Se espera que los arbolitos del ensemble óptimo sean más overfitteros (ej., menor minsplit como 20, y mayor maxdepth como 10) que los de un árbol individual.\n")

format(Sys.time(), "%a %b %d %X %Y")

# --- Fin del Código R ---



---

